**Problem:**

You are given the following dataset:
1. **Audible Data** : https://1drv.ms/u/s!AiqdXCxPTydhoog8ckLN-6Cw55fzIg?e=EWgZ5d

Your task is to:
- Find the problems with the datasets.
- Define the Data Quality Dimensions.
- Try to clean the datasets.

In [240]:
import numpy as np
import pandas as pd

In [ ]:
auidable = pd.read_csv('datasets/audible_uncleaned.csv')
auidable.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00
...,...,...,...,...,...,...,...,...
87484,Last Days of the Bus Club,Writtenby:ChrisStewart,Narratedby:ChrisStewart,7 hrs and 34 mins,09-03-17,English,Not rated yet,596.00
87485,The Alps,Writtenby:StephenO'Shea,Narratedby:RobertFass,10 hrs and 7 mins,21-02-17,English,Not rated yet,820.00
87486,The Innocents Abroad,Writtenby:MarkTwain,Narratedby:FloGibson,19 hrs and 4 mins,30-12-16,English,Not rated yet,938.00
87487,A Sentimental Journey,Writtenby:LaurenceSterne,Narratedby:AntonLesser,4 hrs and 8 mins,23-02-11,English,Not rated yet,680.00


**audible uncleand**
1. The `Author` column contains values with a "Writtenby:" prefix, such as "Writtenby:RickRiordan," which is not a standard format for author names, representing a validity issue categorized as a formatting problem.

2. The `Narrator` column includes values with a "Narratedby:" prefix, such as "Narratedby:BillLobely," which is inconsistent with standard narrator name formats, representing a validity issue categorized as a formatting problem.

3. The `Time` column stores duration values as text in English, such as "2 hrs and 20 mins," and is recorded as an object data type, making it unsuitable for numerical analysis, representing a validity issue categorized as a formatting and data type problem.

4. The `Stars` column combines rating and total ratings in a single field, such as "4.5 out of 5 stars (100 ratings)," or contains text like "Not rated yet," which is inconsistent and non-numeric, representing a validity issue categorized as a formatting and data structure problem.

5. The `Price` column is stored as an object data type, likely due to currency symbols or text entries like "$9.99" or "Free," instead of a numeric float type required for calculations, representing a validity issue categorized as a data type problem.

6. The `Price` column contains a mix of formats, with some values in numeric format like "9.99" and others potentially including currency symbols or text, leading to inconsistent data representation, representing a validity issue categorized as a formatting problem.
 

In [242]:
# corret author column
auidable['author']=auidable['author'].str.replace('Writtenby:','').str.strip()


In [243]:
# correct narroter column 
auidable['narrator']=auidable['narrator'].str.replace('Narratedby:','').str.strip()

In [244]:
# convert time to mins in int dtype
import re




# Step 2: Define function to convert text or numeric duration to minutes
def convert_time_to_minutes(time_val):
    if pd.isna(time_val):  # Handle missing values
        return np.nan
    
    # Handle numeric inputs (int or float, assumed to be minutes)
    if isinstance(time_val, (int, float)):
        return float(time_val)
    
    # Handle string inputs
    try:
        time_str = str(time_val).lower().replace('and', '').strip()
        
        # Initialize hours and minutes
        hours, minutes = 0, 0
        
        # Extract hours (number before 'hr' or 'hrs')
        hr_match = re.search(r'(\d+)\s*hr', time_str)
        hours = int(hr_match.group(1)) if hr_match else 0
        
        # Extract minutes (number before 'min' or 'mins')
        min_match = re.search(r'(\d+)\s*min', time_str)
        minutes = int(min_match.group(1)) if min_match else 0
        
        # Return total minutes
        return hours * 60 + minutes
    except (ValueError, AttributeError):
        # Return NaN for malformed inputs (e.g., 's 20 mins')
        return np.nan

    

# Step 3: Apply conversion to create new column
auidable['time'] = auidable['time'].apply(convert_time_to_minutes)



In [245]:
# change astype
auidable['time']=auidable['time'].astype(int)

In [246]:
# date to datetime
auidable['releasedate']=pd.to_datetime(auidable['releasedate'])

C:\Users\HP\AppData\Local\Temp\ipykernel_9284\2899656558.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  auidable['releasedate']=pd.to_datetime(auidable['releasedate'])


In [247]:
# change rating into only given rating and total rating 
import pandas as pd
import re

# Assuming your dataframe is named 'df'
auidable['star_rating'] = auidable['stars'].str.extract(r'(\d+\.?\d*) out of 5 stars')  # Extracts the star rating
auidable['num_ratings'] = auidable['stars'].str.extract(r'(\d+) ratings')  # Extracts the number of ratings

# Convert the new columns to numeric types
auidable['star_rating'] = pd.to_numeric(auidable['star_rating'])
auidable['num_ratings'] = pd.to_numeric(auidable['num_ratings'])


In [248]:
auidable.fillna(0)

,name,author,narrator,time,releasedate,language,stars,price,star_rating,num_ratings
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,140,2008-04-08,English,5 out of 5 stars34 ratings,468.00,5.0,34.0
1,The Burning Maze,RickRiordan,RobbieDaymond,788,2018-01-05,English,4.5 out of 5 stars41 ratings,820.00,4.5,41.0
2,The Deep End,JeffKinney,DanRussell,123,2020-06-11,English,4.5 out of 5 stars38 ratings,410.00,4.5,38.0
3,Daughter of the Deep,RickRiordan,SoneelaNankani,676,2021-05-10,English,4.5 out of 5 stars12 ratings,615.00,4.5,12.0
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,600,2010-01-13,English,4.5 out of 5 stars181 ratings,820.00,4.5,181.0
...,...,...,...,...,...,...,...,...,...,...
87484,Last Days of the Bus Club,ChrisStewart,ChrisStewart,454,2017-09-03,English,Not rated yet,596.00,0.0,0.0
87485,The Alps,StephenO'Shea,RobertFass,607,2017-02-21,English,Not rated yet,820.00,0.0,0.0
87486,The Innocents Abroad,MarkTwain,FloGibson,1144,2016-12-30,English,Not rated yet,938.00,0.0,0.0
87487,A Sentimental Journey,LaurenceSterne,AntonLesser,248,2011-02-23,English,Not rated yet,680.00,0.0,0.0


In [249]:
auidable['star_rating'] = auidable['star_rating'].fillna(0)
auidable['num_ratings'] = auidable['num_ratings'].fillna(0).astype(int)
auidable.drop(columns={'stars'},inplace=True)

In [250]:
auidable.isnull().sum()

name           0
author         0
narrator       0
time           0
releasedate    0
language       0
price          0
star_rating    0
num_ratings    0
dtype: int64

In [251]:
# Step 1: Replace "Free" and other non-numeric values
auidable['price'] = auidable['price'].replace(['Free', 'N/A', 'Not Available'], '0')

# Step 2: Remove $ and commas
auidable['price'] = auidable['price'].str.replace('[$,]', '', regex=True)

# Step 3: Convert to float (invalid entries become NaN)
auidable['price'] = pd.to_numeric(auidable['price'], errors='coerce')

# Step 4: Fill NaN if needed (optional)
auidable['price'] = auidable['price'].fillna(0)  # Or another default value

In [252]:
auidable['price']=auidable['price'].astype(int)

In [253]:
auidable['price']=auidable['price'].astype(int)

In [254]:
auidable['star_rating'].astype(float)

0        5.0
1        4.5
2        4.5
3        4.5
4        4.5
        ... 
87484    0.0
87485    0.0
87486    0.0
87487    0.0
87488    0.0
Name: star_rating, Length: 87489, dtype: float64